<a href="https://colab.research.google.com/github/CAU-CAPSTONE-2-Friendly-Keyboard/Models/blob/main/%08BadtextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283859 sha256=a01daf4e7fd9392bf02b64b987b16da523bc1a6e0f4419630c1c3ebc5de59ac4
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [2]:
from lime import lime_text
from lime.lime_text import LimeTextExplainer

In [38]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
import numpy as np

In [39]:
explainer = LimeTextExplainer(class_names=['bad','ok'])

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('JminJ/kcElectra_base_Bad_Sentence_Classifier')
tokenizer = AutoTokenizer.from_pretrained('JminJ/kcElectra_base_Bad_Sentence_Classifier')


In [6]:
classifier = pipeline("text-classification", model=model,tokenizer=tokenizer)

In [44]:
test_text = "너 이 새끼야 닥쳐봐"

In [40]:
def mk_predlist(inps):
  result = []
  for inp in inps:
    output = classifier(inp)[0]
    if output['label'] == 'bad_sen':
      result.append([output['score'], 1-output['score']])
    else:
      result.append([1-output['score'], output['score']])
  return np.array(result)

In [24]:
classifier(test_text)[0]

{'label': 'bad_sen', 'score': 0.9220901131629944}

In [43]:
exp.as_list()

[('닥쳐', -0.6122251695282325),
 ('야', -0.10124293419778817),
 ('너', -0.048373352511858383)]

In [45]:
if classifier(test_text)[0]['label'] == 'bad_sen':
  print('비속어 있음')
  exp = explainer.explain_instance(test_text, mk_predlist,top_labels=3)
  print(exp.as_list())
else:
  print('비속어 없음')


[('새끼야', -0.29051266893832417), ('닥쳐봐', -0.22468180441935537), ('이', -0.15621329981568058), ('너', -0.14162732833961472)]
